In [ ]:
!pip install transformers torch -qqq

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `newone` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [ ]:
!huggingface-cli whoami

Uomnaa


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd

# Path to the dataset file
file_path = '/content/train.csv'

# Read the first 2000 rows of the CSV file into a DataFrame
train_data = pd.read_csv(file_path, engine='python', on_bad_lines='skip', nrows=2000)

# Inspect the first few rows
print(train_data.head())
print(train_data.info())


  essay_id                                          full_text  score
0  000d118  Many people have car where they live. The thin...      3
1  000fe60  I am a scientist at NASA that is discussing th...      3
2  001ab80  People always wish they had the same technolog...      4
3  001bdc0  We all heard about Venus, the planet without a...      4
4  002ba53  Dear, State Senator\n\nThis is a letter to arg...      3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   2000 non-null   object
 1   full_text  2000 non-null   object
 2   score      2000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.0+ KB
None


In [ ]:
print(train_data['score'].unique())


[3 4 2 1 5 6]


In [ ]:
train_data['score'] -= 1  # Normalize labels to [0, 5]


In [ ]:
print(train_data.head())
print(train_data.info())
print(train_data.isnull().sum())


  essay_id                                          full_text  score
0  000d118  Many people have car where they live. The thin...      2
1  000fe60  I am a scientist at NASA that is discussing th...      2
2  001ab80  People always wish they had the same technolog...      3
3  001bdc0  We all heard about Venus, the planet without a...      3
4  002ba53  Dear, State Senator\n\nThis is a letter to arg...      2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   2000 non-null   object
 1   full_text  2000 non-null   object
 2   score      2000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.0+ KB
None
essay_id     0
full_text    0
score        0
dtype: int64


In [ ]:
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

train_data['cleaned_full_text'] = train_data['full_text'].apply(preprocess_text)
train_data['essay_word_count'] = train_data['cleaned_full_text'].apply(lambda x: len(x.split()))

print("Data after Preprocessing:")
print(train_data[['full_text', 'cleaned_full_text','essay_word_count']].head())

Data after Preprocessing:
                                           full_text  \
0  Many people have car where they live. The thin...   
1  I am a scientist at NASA that is discussing th...   
2  People always wish they had the same technolog...   
3  We all heard about Venus, the planet without a...   
4  Dear, State Senator\n\nThis is a letter to arg...   

                                   cleaned_full_text  essay_word_count  
0  many people have car where they live. the thin...               498  
1  i am a scientist at nasa that is discussing th...               332  
2  people always wish they had the same technolog...               550  
3  we all heard about venus, the planet without a...               451  
4  dear, state senator\n\nthis is a letter to arg...               373  


https://drlee.io/step-by-step-guide-fine-tuning-metas-llama-3-2-1b-model-f1262eda36c8

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig


# Load the model and tokenizer
model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=6)
# Set the padding token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
max_length = train_data['essay_word_count'].max()
print(f"Max sequence length in dataset: {max_length}")

Max sequence length in dataset: 1036


In [ ]:
def tokenize_data(df, tokenizer, max_length):
    tokenized_inputs = tokenizer(
        list(df["cleaned_full_text"]),  # Use cleaned_full_text
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    # Treat scores as integers or categorical class labels
    tokenized_labels = torch.tensor(df["score"].values)  # Directly use score column as labels

    return tokenized_inputs, tokenized_labels

tokenized_inputs, tokenized_labels = tokenize_data(train_data, tokenizer, max_length)

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Instantiate the dataset
dataset = CustomDataset(tokenized_inputs, tokenized_labels)

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator
import os
import torch
import gc

# Set environment variables for memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Accelerator with CPU Offloading
accelerator = Accelerator()

# Disable caching in model config
model.config.use_cache = False  # Disable use_cache for gradient checkpointing

# Updated Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/checkpoints",
    eval_strategy="no",  # Replace evaluation_strategy
    learning_rate=3e-5,
    per_device_train_batch_size=2,  # Reduce batch size
    gradient_accumulation_steps=8,  # Aggressively increase accumulation
    num_train_epochs=3,
    weight_decay=0.4,
    fp16=True,  # Enable mixed precision
    logging_dir="/content/drive/MyDrive/logs",
    logging_steps=10,
    save_total_limit=2,
    save_strategy="steps",
    save_steps=200,
    gradient_checkpointing=True,  # Gradient checkpointing to save memory
    optim="adafactor",  # Use memory-efficient optimizer
)



# Trainer setup with CPU offloading
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)


trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,2.889500
20,3.953200
30,3.796900
40,2.008700
50,1.701500
60,1.537200
70,1.631000
80,1.495300
90,1.315700
100,1.416800


Step,Training Loss
10,2.889500
20,3.953200
30,3.796900
40,2.008700
50,1.701500
60,1.537200
70,1.631000
80,1.495300
90,1.315700
100,1.416800


TrainOutput(global_step=375, training_loss=1.1687684535980225, metrics={'train_runtime': 4508.2042, 'train_samples_per_second': 1.331, 'train_steps_per_second': 0.083, 'total_flos': 3.62949156864e+16, 'train_loss': 1.1687684535980225, 'epoch': 3.0})

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# Path to the dataset file
file_path = '/content/train.csv'

# Read rows 1000 to 2000 from the CSV file into a DataFrame (for validation dataset)
validation_dataset = pd.read_csv(file_path, engine='python', on_bad_lines='skip', nrows=100)

# Assuming you have already defined preprocess_text function
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

# Apply preprocessing to the validation dataset
validation_dataset['cleaned_full_text'] = validation_dataset['full_text'].apply(preprocess_text)

# Now, split the validation dataset and tokenize it
tokenized_val_inputs, tokenized_val_labels = tokenize_data(validation_dataset, tokenizer, max_length)

# Create the validation dataset using the CustomDataset class
validation_dataset = CustomDataset(tokenized_val_inputs, tokenized_val_labels)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a DataLoader for the validation dataset (this will use the entire validation data)
validation_dataloader = DataLoader(validation_dataset, batch_size=1, collate_fn=data_collator)


In [ ]:
def compute_accuracy(validation_dataloader, model, device):
    # Ensure the model is in evaluation mode
    model.eval()

    correct_predictions = 0
    total_predictions = 0

    # Loop through the dataset
    for batch in validation_dataloader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Predict
        with torch.no_grad():
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
            labels = batch["labels"]

        # Update metrics
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    # Compute accuracy
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Usage
accuracy = compute_accuracy(validation_dataloader, model, device)
print(f"Accuracy on the training dataset: {accuracy:.2f}%")


Accuracy on the training dataset: 91.00%


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# Path to the dataset file
file_path = '/content/train.csv'

# Read rows 1000 to 2000 from the CSV file into a DataFrame (for validation dataset)
validation_dataset = pd.read_csv(file_path, engine='python', on_bad_lines='skip', skiprows=range(1, 2000), nrows=100)

# Assuming you have already defined preprocess_text function
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

# Apply preprocessing to the validation dataset
validation_dataset['cleaned_full_text'] = validation_dataset['full_text'].apply(preprocess_text)

# Now, split the validation dataset and tokenize it
tokenized_val_inputs, tokenized_val_labels = tokenize_data(validation_dataset, tokenizer, max_length)

# Create the validation dataset using the CustomDataset class
validation_dataset = CustomDataset(tokenized_val_inputs, tokenized_val_labels)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a DataLoader for the validation dataset (this will use the entire validation data)
validation_dataloader = DataLoader(validation_dataset, batch_size=1, collate_fn=data_collator)


In [ ]:
def compute_accuracy(validation_dataloader, model, device):
    # Ensure the model is in evaluation mode
    model.eval()

    correct_predictions = 0
    total_predictions = 0

    # Loop through the dataset
    for batch in validation_dataloader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Predict
        with torch.no_grad():
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
            labels = batch["labels"]

        # Update metrics
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    # Compute accuracy
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Usage
accuracy = compute_accuracy(validation_dataloader, model, device)
print(f"Accuracy on the validation dataset: {accuracy:.2f}%")


Accuracy on the validation dataset: 69.00%


In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 34
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 34
Input Text: the planet that some call the "evening star," venus, is the closest planet to earth at points in time. despite the hostile and dangerous environment, the close proximity and sustainable innovations outweigh the challenges and open up many doors for research of one of earth's neighbors. long ago, venus was most likely the most similar planet to earth in the solar system.

in looking at the development and changes that venus has gone through, the planet's past may turn into earth's future. though right now the atmosphere contains "almost <num> percent carbon dioxide blankets," and the surface averages over "<num> degrees fahrenheit," it has features found on earth as well. these include "rocky sediment...valleys, mountains, and craters." analyzing this enviroment more closely would allow for a deeper understanding of venus, and may fufill the curiosity of scientists wondering what earth's "twin" truly is like.

as the conditions of the planet were described brutally

In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 55
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 55
Input Text: paragraph <num> implies that, "the electoral college processs consist of the selection of the electors, the meeting of the electors where they vote for president and vice president, and the counting of the eclectoral votes by congress." because of that i pesonally think that we should not keep the electoral college, because people want to choose who they want to be preseident they don't want to rely on someone else to choose for them, that may even end up betraying them,and also when you vote you think you are actually voting for a presidential candidate but you're actually voting for a state of electors, the whole point on voting is to choose who you want. i think it's unfair to everyone just like

source <num>: the indefensible electoral college: why even the best-laid defenses of the system are wrong said in paragraph <num>

"it's official: the electoral college is unfair, outdated, and irrational, and also in

source <num>: in defense of the electoral college:

In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 41
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 41
Input Text: in the article "driverless cars are coming"

the author covers many points and facts about driverless cars. the author speaks about the advantages and disadvantages of the introduction of these cars. the author also speaks about the major companies making these cars from google to nissan. nontheless driverless cars could prove to be unsafe, epensive, and have to many negative possibilites. these negative tradeoffs out weigh the possitive tradeoffs thus showing that introducing driverless cars would do more harm then good.

the article states "most driving laws focus on keeping drivers, passengers, and pedestrians safe..." the introduction of driverless cars would make this law harder to keep. this would be because the burden to be safe would no longer be in human hands but in the hands of a cumputer and some sensors. we have all had computers glitch or take a while to repond. it happens all the time. but then the safty of human life is in the hand of a computer it

In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 56
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 56
Input Text: if i was working for nasa and l had an argument with a person that believes that ''the face'' was created by aliens, i would say that it is just a simple landform that was made on mars. i say this because if i were to work for nasa i would look back at data that will help me defend my answer, that ''the face'' was not created by aliens. for all we know it could just be a mountain that has ''eyes'', ''a mouth'', ''and a nose'' due to shadowing.

we human beings really have no clue if there are aliens or not, we find ''clues'' and believe there are but, in my opinion there are none. in the article mr. jim gavin said ''it reminds me most of middle butte in the snake river plain of idaho''. this is what i would say if i had a discussion with a person who believed ''the face'' was created by aliens.   
Actual Label: 2
Predicted Label: 2


In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 3
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 3
Input Text: in the following essay i'm going to talk about how this new technology called facial action coding system or facs help us to identify human emotions. i think that the use of this new technology will be completely valuable to read the emotional expressions of students in a classroom. will the use of this new technology be helpful to the teacher? will this techonolgy be use just inside of classrooms?

in different situations know how a person feels is important, not just because the person will probably need help but to understand the emotion the person is feeling. i believe that this kind of new technology will be more than helpful inside of the classrooms at schools because the technology can tell the teacher wether the students are learning or not; can also tell the teacher if he needs to improve himself to make the class even interesting instead of bored. i think this kind of technology has to be used not just inside of schools but can also be used at important p

In [ ]:
def display_prediction(index, validation_dataloader, model, tokenizer, device):

    model.eval()

    # Retrieve the specific sample
    for i, batch in enumerate(validation_dataloader):
        if i == index:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Predict
            with torch.no_grad():
                outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
                predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
                labels = batch["labels"]

            # Decode input text for display
            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)

            # Display results
            print(f"Index: {index}")
            print(f"Input Text: {input_text}")
            print(f"Actual Label: {labels.item()}")
            print(f"Predicted Label: {predictions.item()}")
            break
    else:
        print(f"Index {index} out of range for the validation dataset.")


index_to_check = 34
display_prediction(index_to_check, validation_dataloader, model, tokenizer, device)


Index: 34
Input Text: the planet that some call the "evening star," venus, is the closest planet to earth at points in time. despite the hostile and dangerous environment, the close proximity and sustainable innovations outweigh the challenges and open up many doors for research of one of earth's neighbors. long ago, venus was most likely the most similar planet to earth in the solar system.

in looking at the development and changes that venus has gone through, the planet's past may turn into earth's future. though right now the atmosphere contains "almost <num> percent carbon dioxide blankets," and the surface averages over "<num> degrees fahrenheit," it has features found on earth as well. these include "rocky sediment...valleys, mountains, and craters." analyzing this enviroment more closely would allow for a deeper understanding of venus, and may fufill the curiosity of scientists wondering what earth's "twin" truly is like.

as the conditions of the planet were described brutally

In [ ]:
def display_predictions_summary(validation_dataloader, model, device):
    # Ensure the model is in evaluation mode
    model.eval()

    # Loop through the dataset
    for index, batch in enumerate(validation_dataloader):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Predict
        with torch.no_grad():
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            predictions = torch.argmax(outputs.logits, dim=1) + 1  # Shift predictions to the range 1-6
            labels = batch["labels"]

        # Print results in "Actual -> Predicted" format
        print(f"Index: {index} | Actual: {labels.item()} -> Predicted: {predictions.item()}")

# Usage
display_predictions_summary(validation_dataloader, model, device)


Index: 0 | Actual: 3 -> Predicted: 3
Index: 1 | Actual: 3 -> Predicted: 3
Index: 2 | Actual: 2 -> Predicted: 2
Index: 3 | Actual: 2 -> Predicted: 3
Index: 4 | Actual: 4 -> Predicted: 4
Index: 5 | Actual: 4 -> Predicted: 4
Index: 6 | Actual: 3 -> Predicted: 2
Index: 7 | Actual: 4 -> Predicted: 3
Index: 8 | Actual: 3 -> Predicted: 3
Index: 9 | Actual: 1 -> Predicted: 1
Index: 10 | Actual: 2 -> Predicted: 2
Index: 11 | Actual: 2 -> Predicted: 2
Index: 12 | Actual: 2 -> Predicted: 2
Index: 13 | Actual: 3 -> Predicted: 3
Index: 14 | Actual: 2 -> Predicted: 2
Index: 15 | Actual: 4 -> Predicted: 4
Index: 16 | Actual: 5 -> Predicted: 4
Index: 17 | Actual: 2 -> Predicted: 3
Index: 18 | Actual: 4 -> Predicted: 4
Index: 19 | Actual: 3 -> Predicted: 3
Index: 20 | Actual: 3 -> Predicted: 3
Index: 21 | Actual: 3 -> Predicted: 2
Index: 22 | Actual: 1 -> Predicted: 1
Index: 23 | Actual: 3 -> Predicted: 3
Index: 24 | Actual: 4 -> Predicted: 4
Index: 25 | Actual: 3 -> Predicted: 3
Index: 26 | Actual: 2 

In [ ]:
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from transformers import AutoModel, AutoTokenizer

model.push_to_hub("llama3.2_essayscoring")
tokenizer.push_to_hub("llama3.2_essayscoring")


model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Uomnaa/llama3.2_essayscoring/commit/1747529a9c5376c798f34e728cb28d740e59bc61', commit_message='Upload tokenizer', commit_description='', oid='1747529a9c5376c798f34e728cb28d740e59bc61', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Uomnaa/llama3.2_essayscoring', endpoint='https://huggingface.co', repo_type='model', repo_id='Uomnaa/llama3.2_essayscoring'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "Uomnaa/llama3.2_essayscoring"  # Replace with your model name

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Path to the dataset file
file_path = '/content/train.csv'

# Read rows 1000 to 2000 from the CSV file into a DataFrame (for validation dataset)
validation_dataset = pd.read_csv(file_path, engine='python', on_bad_lines='skip', skiprows=range(1, 2000), nrows=100)

# Preprocessing function for text
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

# Apply preprocessing to the validation dataset
validation_dataset['cleaned_full_text'] = validation_dataset['full_text'].apply(preprocess_text)

# Prompt engineering: Add context or task instructions to the text
def apply_prompt_engineering(text):
    prompt = "Evaluate the following essay and provide a score from 0 to 5: "
    return prompt + text

validation_dataset['prompted_text'] = validation_dataset['cleaned_full_text'].apply(apply_prompt_engineering)

# Custom dataset class for tokenized data
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Tokenize data with padding and truncation
def tokenize_data(dataset, tokenizer, max_length):
    tokenized_inputs = tokenizer(
        dataset['prompted_text'].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    labels = dataset['score'].tolist()  # Adjust column name if needed
    return tokenized_inputs, labels

# Tokenize validation data
max_length = 1036
tokenized_val_inputs, tokenized_val_labels = tokenize_data(validation_dataset, tokenizer, max_length)

# Create the validation dataset
validation_dataset = CustomDataset(tokenized_val_inputs, tokenized_val_labels)

# Data collator to dynamically pad inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create DataLoader for validation
validation_dataloader = DataLoader(validation_dataset, batch_size=1, collate_fn=data_collator)

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in tqdm(validation_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        actuals.extend(labels.cpu().numpy())

# Adjust predictions to match the actual scoring range (add 1)
predictions = [pred + 1 for pred in predictions]

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(actuals, predictions)

print(f"Validation Accuracy: {accuracy:.2f}")


100%|██████████| 100/100 [01:05<00:00,  1.53it/s]

Validation Accuracy: 0.70


**####################################################################################**

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Path to the dataset file
file_path = '/content/train.csv'

# Read rows 1000 to 2000 from the CSV file into a DataFrame (for validation dataset)
validation_dataset = pd.read_csv(file_path, engine='python', on_bad_lines='skip', skiprows=range(1, 2000), nrows=100)

# Preprocessing function for text
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

# Apply preprocessing to the validation dataset
validation_dataset['cleaned_full_text'] = validation_dataset['full_text'].apply(preprocess_text)

# Prompt engineering: Add context or task instructions to the text
def apply_prompt_engineering(text):
    prompt = '''You are an expert essay evaluator.
    Your task is to analyze the following essay based on its coherence, grammar, structure, relevance to the topic, and overall quality.
    Provide a score from 1 to 6, where 1 indicates poor performance and 6 indicates excellent performance.
    Focus on evaluating the clarity of ideas, grammatical correctness, and the logical flow of arguments.'''
    return prompt + text

validation_dataset['prompted_text'] = validation_dataset['cleaned_full_text'].apply(apply_prompt_engineering)

# Custom dataset class for tokenized data
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Tokenize data with padding and truncation
def tokenize_data(dataset, tokenizer, max_length):
    tokenized_inputs = tokenizer(
        dataset['prompted_text'].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    labels = dataset['score'].tolist()  # Adjust column name if needed
    return tokenized_inputs, labels

# Tokenize validation data
max_length = 1036
tokenized_val_inputs, tokenized_val_labels = tokenize_data(validation_dataset, tokenizer, max_length)

# Create the validation dataset
validation_dataset = CustomDataset(tokenized_val_inputs, tokenized_val_labels)

# Data collator to dynamically pad inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create DataLoader for validation
validation_dataloader = DataLoader(validation_dataset, batch_size=1, collate_fn=data_collator)

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in tqdm(validation_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        actuals.extend(labels.cpu().numpy())

# Adjust predictions to match the actual scoring range (add 1)
predictions = [pred + 1 for pred in predictions]

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(actuals, predictions)

print(f"Validation Accuracy: {accuracy:.2f}")


100%|██████████| 100/100 [01:05<00:00,  1.53it/s]

Validation Accuracy: 0.71


**############################################################**